In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tongue-all/Tongue_Features.xlsx
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709793682402.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709982568772.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709708320994.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709800877068.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709712334590.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709971033212.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709617694248.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709635624336.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Palette Yolo Cropped/1709701638747.jpg
/kaggle/input/tongue-all/Tongue Palette Yolo Cropped/Tongue Pal

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
# Read Excel file and select needed columns
df = pd.read_excel('/kaggle/input/tongue-all/Tongue_Features.xlsx')
df = df[['Image_Name', 'Hb Value']]

In [5]:
# Split data into train (approx. 70%), validation (approx. 15%), and test (15%)
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.176, random_state=42)

In [6]:
# Function to load and preprocess images
def load_image_and_label(image_file, label):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize for ResNet50
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# Function to create TensorFlow datasets
def create_tf_dataset(df, images_folder, batch_size=16, shuffle=True):
    image_paths = [os.path.join(images_folder, img) for img in df['Image_Name'].values]
    labels = df['Hb Value'].values.astype(np.float32)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Define your image folder path and create datasets
images_folder = "/kaggle/input/tongue-all/Tongue Masked/Tongue Masked"
train_ds = create_tf_dataset(train_df, images_folder, batch_size=16, shuffle=True)
val_ds   = create_tf_dataset(val_df,   images_folder, batch_size=16, shuffle=False)
test_ds  = create_tf_dataset(test_df,  images_folder, batch_size=16, shuffle=False)

In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load ResNet50 base (without top) with pretrained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Build the regression head on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', name='feature_dense')(x)  # Named layer for later feature extraction
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Linear activation for regression

# Construct the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_absolute_error', metrics=['mae'])

# Train the new head (freeze base model)
history = model.fit(train_ds, validation_data=val_ds, epochs=50)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 501ms/step - loss: 8.8977 - mae: 8.8977 - val_loss: 2.5562 - val_mae: 2.5562
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.1647 - mae: 2.1647 - val_loss: 1.8104 - val_mae: 1.8104
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 1.7461 - mae: 1.7461 - val_loss: 1.6619 - val_mae: 1.6619
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 1.6729 - mae: 1.6729 - val_loss: 1.6495 - val_mae: 1.6495
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 1.6669 - mae: 1.6669 - val_loss: 1.6642 - val_mae: 1.6642
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 1.6461 - mae: 1.6461 - val_loss: 1.6486 - val_mae: 1.6486
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 1.6542 - mae: 1.6542 - val_loss: 1.6499 - val_mae: 1.6499
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 1.5813 - mae: 1.5813 - val_loss: 1.6473 - val_mae: 1.6473
Epoch 9/50


In [8]:
# Unfreeze the last 20 layers of the base model for fine tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate for fine tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='mean_absolute_error', metrics=['mae'])

# Fine tune the model further
history_ft = model.fit(train_ds, validation_data=val_ds, epochs=100)

# Optionally, evaluate the fine tuned model on the test dataset
test_mae = model.evaluate(test_ds)
print("Test MAE after fine tuning:", test_mae)


Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 24s 527ms/step - loss: 3.4711 - mae: 3.4711 - val_loss: 1.7933 - val_mae: 1.7933
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 1.5994 - mae: 1.5994 - val_loss: 1.7847 - val_mae: 1.7847
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 1.3970 - mae: 1.3970 - val_loss: 1.6914 - val_mae: 1.6914
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 1.5251 - mae: 1.5251 - val_loss: 1.6673 - val_mae: 1.6673
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 1.4830 - mae: 1.4830 - val_loss: 1.9738 - val_mae: 1.9738
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.3473 - mae: 1.3473 - val_loss: 2.5283 - val_mae: 2.5283
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 1.4114 - mae: 1.4114 - val_loss: 2.9534 - val_mae: 2.9534
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 1.2884 - mae: 1.2884 - val_loss: 3.2894 - val_mae: 3.2894
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - 

In [9]:
# Create a feature extractor model that outputs the activations of the "feature_dense" layer
feature_extractor = Model(inputs=model.input, outputs=model.get_layer("feature_dense").output)

# Define a helper function to extract features from a TF dataset
def extract_features(dataset, extractor_model):
    features_list = []
    labels_list = []
    for batch_images, batch_labels in dataset:
        features = extractor_model.predict(batch_images)
        features_list.append(features)
        labels_list.append(batch_labels.numpy())
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.concatenate(labels_list, axis=0)
    return features_array, labels_array

# Extract features and labels from each dataset
train_features, train_labels = extract_features(train_ds, feature_extractor)
val_features, val_labels     = extract_features(val_ds, feature_extractor)
test_features, test_labels   = extract_features(test_ds, feature_extractor)

print("Train features shape:", train_features.shape)
print("Validation features shape:", val_features.shape)
print("Test features shape:", test_features.shape)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━

In [10]:
!pip install pymrmr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp310-cp310-linux_x86_64.whl size=390767 sha256=51b2049cd7c8786d33a31fdbf302095e6fd007301713daded506ea55203e767e
  Stored in directory: /root/.cache/pip/wheels/46/ae/55/4a2479c5f0de7eb363fe970cb18e4a750e03e4e63b1b5c2005
Successfully built pymrmr


In [11]:
import pandas as pd
import numpy as np
import pymrmr

# Assume 'train_features' and 'train_labels' are from the feature extractor
# Create a DataFrame with columns f0, f1, ..., and the target "Hb Value"
train_feat_df = pd.DataFrame(train_features, columns=[f"f{i}" for i in range(train_features.shape[1])])
train_feat_df['Hb Value'] = train_labels

# Discretize each feature column into 10 quantile bins.
for col in train_feat_df.columns[:-1]:  # Exclude the target column
    # qcut will bin the continuous values into 10 categories.
    train_feat_df[col] = pd.qcut(train_feat_df[col], q=10, duplicates='drop').cat.codes

# Discretize the target as well.
train_feat_df['Hb Value'] = pd.qcut(train_feat_df['Hb Value'], q=10, duplicates='drop').cat.codes


In [12]:
# Define the number of top features you wish to select
num_top_features = 100

# Run mRMR to select features
selected_features = pymrmr.mRMR(train_feat_df, 'MIQ', num_top_features)
print("Selected features:", selected_features)

# Convert feature names (e.g., "f23") to column indices
selected_indices = []

for feat in selected_features:
    # Ensure the feature name starts with 'f' and is followed by digits
    if feat.startswith('f') and feat[1:].isdigit():
        selected_indices.append(int(feat[1:]))
    else:
        print(f"Skipping non-standard feature: {feat}")

print("Selected feature indices:", selected_indices)


Selected features: ['f530', 'f189', 'f961', 'f624', 'f155', 'f585', 'f83', 'f897', 'f753', 'f16', 'f364', 'f954', 'f384', 'f317', 'f573', 'f534', 'f803', 'f995', 'f453', 'f641', 'f709', 'f270', 'f469', 'f971', 'f780', 'f659', 'f413', 'f742', 'f365', 'f865', 'f832', 'f843', 'f117', 'f901', 'f265', 'f653', 'f169', 'f538', 'f711', 'f25', 'f296', 'f662', 'f985', 'f104', 'f700', 'f115', 'f681', 'f692', 'f638', 'f234', 'f281', 'f408', 'f763', 'f682', 'f798', 'f1020', 'f915', 'f482', 'f577', 'f776', 'f172', 'f743', 'f397', 'f598', 'f669', 'f595', 'f407', 'f442', 'f174', 'f797', 'f125', 'f79', 'f854', 'f741', 'f48', 'f411', 'f431', 'f313', 'f993', 'f567', 'f1003', 'f548', 'f925', 'f344', 'f295', 'f886', 'f616', 'f31', 'f242', 'f268', 'f800', 'f391', 'f502', 'f685', 'f94', 'f936', 'f237', 'f141', 'f574', 'f826']
Selected feature indices: [530, 189, 961, 624, 155, 585, 83, 897, 753, 16, 364, 954, 384, 317, 573, 534, 803, 995, 453, 641, 709, 270, 469, 971, 780, 659, 413, 742, 365, 865, 832, 843, 

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Slice the continuous features using the selected indices
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Train the regression model on the training set
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train, train_labels)

# Predict on the validation set
val_preds = regressor.predict(X_val)
val_mae = mean_absolute_error(val_labels, val_preds)
print("Validation MAE:", val_mae)

# Predict on the test set
test_preds = regressor.predict(X_test)
test_mae = mean_absolute_error(test_labels, test_preds)
print("Test MAE:", test_mae)


Validation MAE: 1.747555514782194
Test MAE: 1.8180794024467473


In [14]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Slice the continuous features using the selected indices from mRMR
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Initialize XGBoost regressor with typical hyperparameters
xgb_regressor = xgb.XGBRegressor(
    objective='reg:squarederror',  # For regression tasks
    n_estimators=200,               # Number of trees
    learning_rate=0.05,             # Step size shrinkage
    max_depth=6,                    # Maximum depth of a tree
    random_state=42
)

# Train the XGBoost model, using the validation set for early stopping
xgb_regressor.fit(
    X_train, 
    train_labels, 
    eval_set=[(X_val, val_labels)],
    early_stopping_rounds=10,
    verbose=True
)

# Make predictions on the validation and test sets
val_preds = xgb_regressor.predict(X_val)
test_preds = xgb_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE) for validation and test sets
val_mae = mean_absolute_error(val_labels, val_preds)
test_mae = mean_absolute_error(test_labels, test_preds)

print("Validation MAE:", val_mae)
print("Test MAE:", test_mae)


[0]	validation_0-rmse:1.92142
[1]	validation_0-rmse:1.92418
[2]	validation_0-rmse:1.92605
[3]	validation_0-rmse:1.93088
[4]	validation_0-rmse:1.93864
[5]	validation_0-rmse:1.94264
[6]	validation_0-rmse:1.94823
[7]	validation_0-rmse:1.95102
[8]	validation_0-rmse:1.95701
[9]	validation_0-rmse:1.96904
Validation MAE: 1.6458623
Test MAE: 1.5826021


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [15]:
!pip install ace_tools

In [16]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

# List of regressors to try
regressors = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42)
}

# Slice the continuous features using the selected indices
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Dictionary to store MAE results
results = {}

# Iterate through each model
for name, model in regressors.items():
    print(f"Training {name}...")
    model.fit(X_train, train_labels)
    
    # Validation Predictions
    val_preds = model.predict(X_val)
    val_mae = mean_absolute_error(val_labels, val_preds)
    
    # Test Predictions
    test_preds = model.predict(X_test)
    test_mae = mean_absolute_error(test_labels, test_preds)
    
    results[name] = {"Validation MAE": val_mae, "Test MAE": test_mae}
    print(f"{name} - Validation MAE: {val_mae:.4f} | Test MAE: {test_mae:.4f}")

# Display results
import pandas as pd
results_df = pd.DataFrame(results).T
import ace_tools as tools; tools.display_dataframe_to_user(name="Regressor Model Performance", dataframe=results_df)


Training RandomForest...
RandomForest - Validation MAE: 1.7476 | Test MAE: 1.8181
Training GradientBoosting...
GradientBoosting - Validation MAE: 1.7213 | Test MAE: 1.7944
Training AdaBoost...
AdaBoost - Validation MAE: 1.7033 | Test MAE: 1.7561
Training LinearRegression...
LinearRegression - Validation MAE: 1.7953 | Test MAE: 1.8011
Training Ridge...
Ridge - Validation MAE: 1.7738 | Test MAE: 1.7853
Training Lasso...
Lasso - Validation MAE: 1.6386 | Test MAE: 1.5861
Training SVR...
SVR - Validation MAE: 1.7711 | Test MAE: 1.7904
Training DecisionTree...
DecisionTree - Validation MAE: 2.0254 | Test MAE: 2.0206
Training KNN...
KNN - Validation MAE: 1.7492 | Test MAE: 1.7775
Training XGBoost...
XGBoost - Validation MAE: 1.8010 | Test MAE: 1.7613
Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7966
[LightGBM] [Info] Number of d

ModuleNotFoundError: No module named 'ace_tools'